## Date and Timestamp Functions

The default format of the PySpark Date is `yyyy-MM-dd`.

In [0]:
dbutils.library.restartPython() # Removes Python state, but some libraries might not work without calling this command.dbutils.restartPython()

#### Load libraries

In [0]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import IntegerType, DateType, StringType, StructType, StructField, ArrayType, MapType, DoubleType
from pyspark.sql.functions import lit, col, expr, when, sum, avg, max, min, mean, count, udf, explode, concat_ws
from pyspark.sql.functions import year, month, dayofmonth, dayofyear, dayofweek, current_date, date_format, to_date, datediff, months_between, trunc, add_months, date_add, date_sub, next_day, weekofyear, current_timestamp, to_timestamp, hour, minute, second, quarter

#### Create Spark session

In [0]:
spark = SparkSession.builder.appName('PySpark Date and Timestamp Functions').getOrCreate()

#### Create Dataframe

In [0]:
data=[
  ('1','2020-02-01'),
  ('2','2019-03-15'),
  ('3','2021-09-28')
]

df = spark.createDataFrame(data,['id','input'])
df.show()

+---+----------+
 id| input|
+---+----------+
 1|2020-02-01|
 2|2019-03-15|
 3|2021-09-28|
+---+----------+

#### current_date()

In [0]:
df.select(current_date().alias('current_date')).show()

+------------+
current_date|
+------------+
 2021-08-12|
 2021-08-12|
 2021-08-12|
+------------+

#### date_format()

In [0]:
df.select(
  col('input'), 
  date_format(col('input'), 'MM-dd-yyyy').alias('date_format') 
).show()

+----------+-----------+
 input|date_format|
+----------+-----------+
2020-02-01| 02-01-2020|
2019-03-15| 03-15-2019|
2021-09-28| 09-28-2021|
+----------+-----------+

#### to_date()

In [0]:
date_df = df.select(
  col('input'), 
  to_date(col('input'), 'yyyy-MM-dd').alias('to_date') 
)

date_df.printSchema()
date_df.show()

root
-- input: string (nullable = true)
-- to_date: date (nullable = true)

+----------+----------+
 input| to_date|
+----------+----------+
2020-02-01|2020-02-01|
2019-03-15|2019-03-15|
2021-09-28|2021-09-28|
+----------+----------+

#### datediff()

In [0]:
df.select(
  col('input'), 
  datediff(current_date(),col('input')).alias('datediff')  
).show()

+----------+--------+
 input|datediff|
+----------+--------+
2020-02-01| 558|
2019-03-15| 881|
2021-09-28| -47|
+----------+--------+

#### months_between()

In [0]:
df.select(
  col('input'), 
  months_between(current_date(),col('input')).alias('months_between')  
).show()

+----------+--------------+
 input|months_between|
+----------+--------------+
2020-02-01| 18.35483871|
2019-03-15| 28.90322581|
2021-09-28| -1.51612903|
+----------+--------------+

#### trunc()

In [0]:
df.select(
  col('input'),
  trunc(col('input'),'Year').alias('Year_Trunc'), 
  trunc(col('input'),'Quarter').alias('Quarter_Trunc'),
  trunc(col('input'),'Month').alias('Month_Trunc')
).show()

+----------+----------+-------------+-----------+
 input|Year_Trunc|Quarter_Trunc|Month_Trunc|
+----------+----------+-------------+-----------+
2020-02-01|2020-01-01| 2020-01-01| 2020-02-01|
2019-03-15|2019-01-01| 2019-01-01| 2019-03-01|
2021-09-28|2021-01-01| 2021-07-01| 2021-09-01|
+----------+----------+-------------+-----------+

#### add_months() , date_add(), date_sub()

In [0]:
df.select(
  col('input'), 
  add_months(col('input'),3).alias ('add_months'), 
  add_months(col('input'),-3).alias('add_months_sub'),
  date_add(col('input'),4).alias('date_add'), 
  date_add(col('input'),-4).alias('date_add_sub'),
  date_sub(col('input'),4).alias('date_sub'),
  date_sub(col('input'),-4).alias('date_sub_add') 
).show()

+----------+----------+--------------+----------+------------+----------+------------+
 input|add_months|add_months_sub| date_add|date_add_sub| date_sub|date_sub_add|
+----------+----------+--------------+----------+------------+----------+------------+
2020-02-01|2020-05-01| 2019-11-01|2020-02-05| 2020-01-28|2020-01-28| 2020-02-05|
2019-03-15|2019-06-15| 2018-12-15|2019-03-19| 2019-03-11|2019-03-11| 2019-03-19|
2021-09-28|2021-12-28| 2021-06-28|2021-10-02| 2021-09-24|2021-09-24| 2021-10-02|
+----------+----------+--------------+----------+------------+----------+------------+

#### year(), quarter(), month(), weekofyear()

In [0]:
df.select(
  col('input'), 
  year(col('input')).alias('year'),
  quarter(col('input')).alias('quarter'),
  month(col('input')).alias('month'), 
  weekofyear(col('input')).alias('weekofyear') 
).show()

+----------+----+-------+-----+----------+
 input|year|quarter|month|weekofyear|
+----------+----+-------+-----+----------+
2020-02-01|2020| 1| 2| 5|
2019-03-15|2019| 1| 3| 11|
2021-09-28|2021| 3| 9| 39|
+----------+----+-------+-----+----------+

#### next_day()

In [0]:
df.select(
  col('input'),
  next_day(col('input'),'Sunday').alias('next_sunday'),
  next_day(col('input'),'Monday').alias('next_monday')
).show()

+----------+-----------+-----------+
 input|next_sunday|next_monday|
+----------+-----------+-----------+
2020-02-01| 2020-02-02| 2020-02-03|
2019-03-15| 2019-03-17| 2019-03-18|
2021-09-28| 2021-10-03| 2021-10-04|
+----------+-----------+-----------+

#### dayofweek(), dayofmonth(), dayofyear()

In [0]:
df.select(
  col('input'),  
  dayofweek(col('input')).alias('dayofweek'), 
  dayofmonth(col('input')).alias('dayofmonth'), 
  dayofyear(col('input')).alias('dayofyear'), 
).show()

+----------+---------+----------+---------+
 input|dayofweek|dayofmonth|dayofyear|
+----------+---------+----------+---------+
2020-02-01| 7| 1| 32|
2019-03-15| 6| 15| 74|
2021-09-28| 3| 28| 271|
+----------+---------+----------+---------+

#### current_timestamp()

In [0]:
df.select(
  current_timestamp().alias('current_timestamp')
).show(1,truncate=False)

+-----------------------+
current_timestamp |
+-----------------------+
2021-08-12 11:25:53.311|
+-----------------------+
only showing top 1 row

#### to_timestamp()

In [0]:
df2 = df.select(
  current_timestamp().alias('current_timestamp'),
  to_timestamp(col('current_timestamp'), 'yyyy-MM-dd HH:mm:ss.SSS').alias('to_timestamp') 
).distinct()

df2.printSchema()
df2.show(truncate=False)

root
-- current_timestamp: timestamp (nullable = false)
-- to_timestamp: timestamp (nullable = true)

+-----------------------+-----------------------+
current_timestamp |to_timestamp |
+-----------------------+-----------------------+
2021-08-12 11:30:20.761|2021-08-12 11:30:20.761|
+-----------------------+-----------------------+

#### hour(), minute(), second()

In [0]:
df2.select(
  col('current_timestamp'), 
  hour(col('current_timestamp')).alias('hour'), 
  minute(col('current_timestamp')).alias('minute'),
  second(col('current_timestamp')).alias('second')
).show(truncate=False)

+-----------------------+----+------+------+
current_timestamp |hour|minute|second|
+-----------------------+----+------+------+
2021-08-12 11:31:57.723|11 |31 |57 |
+-----------------------+----+------+------+

#### The end of the notebook